# Hacking with [Llava-Next](https://llava-vl.github.io/blog/2024-01-30-llava-next/)!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harpreetsahota204/hacking-with-llava-next/blob/main/notebooks/Llava_Next_on_TextVQA.ipynb)

This notebook was created by [Harpreet Sahota](https://twitter.com/DataScienceHarp), Hacker-in-Residence at [Voxel 51](https://voxel51.com/).

In [ ]:
!git clone https://github.com/harpreetsahota204/hacking-with-llava-next.git

In [ ]:
!pip install -r "content/hacking-with-llava-next/requirements.txt"

Let's start by loading the model and processors from the Hugging Face Hub.

We'll try out [llava-v1.6-mistral-7b-hf](https://huggingface.co/llava-hf/llava-v1.6-mistral-7b-hf) and [llava-v1.6-mistral-7b-hf](https://huggingface.co/llava-hf/llava-v1.6-mistral-7b-hf) varieties of these models.

Note that this notebook is run on an A100 from Google Colab Pro+, though you can use a V100 as well.

In [ ]:
import sys

sys.path.append("/content/hacking-with-llava-next/src")

In [ ]:
from utils import load_model_and_processor

mistral_llava, mistral_llava_processor = load_model_and_processor("llava-hf/llava-v1.6-mistral-7b-hf")

vicuna_llava, vicuna_llava_processor = load_model_and_processor("llava-hf/llava-v1.6-vicuna-7b-hf")

In [ ]:
from prompts import MISTRAL_PROMPT, VICUNA_PROMPT

In [ ]:
model_dict = {
    "Mistral": {
        "prompt_template": MISTRAL_PROMPT,
        "model": mistral_llava,
        "processor": mistral_llava_processor
    },
    "Vicuna": {
        "prompt_template": VICUNA_PROMPT,
        "model": vicuna_llava,
        "processor": vicuna_llava_processor
    }
}

# Let's test the models on the following image!


In [ ]:
from utils import ask_question_of_image

In [ ]:
from IPython.display import display, Markdown

image = Image.open("")

questions = [
    "",
    "",
    "",
    "",
    ]

# Loop through each model and question
for model_name, details in model_dict.items():
    for question in questions:
        # Markdown formatted print for model and question
        display(Markdown(f"### Model: **{model_name}**, Question: **{question}**"))

        # Call your function with the current parameters
        ask_question_of_image(
            image=image,
            prompt_template=details["prompt_template"],
            question=question,
            model=details["model"],
            processor=details["processor"]
        )

        # Print a horizontal rule in Markdown for separation
        display(Markdown("---"))

Now, let's test the models on a larger dataset.

Let's download an oldie, but a goodie, [the TextVQA dataset](https://huggingface.co/datasets/textvqa) from Hugging Face. We'll make use of the validation set since we want some answers for evaluation.

To save time, let's just take a small subset of the entire validation set.

In [ ]:
from utils import prepare_dataset

textvqa_val_subset = prepare_dataset(
    dataset_name="textvqa", 
    split="validation", 
    num_samples=500
    )

## And now we can run inference!


In [ ]:
from utils import run_inference_on_dataset

In [ ]:
textvqa_val_subset = run_inference_on_dataset(
    dataset=textvqa_val_subset,
    prompt_template=model_dict["Mistral"]["prompt_template"],
    output_key="mistral_answer",
    model=model_dict["Mistral"]["model"],
    processor=model_dict["Mistral"]["processor"]
    )

In [ ]:
textvqa_val_subset = run_inference_on_dataset(
    dataset=textvqa_val_subset,
    prompt_template=model_dict["Vicuna"]["prompt_template"],
    output_key="vicuna_answer",
    model=model_dict["Vicuna"]["model"],
    processor=model_dict["Vicuna"]["processor"]
    )

# Evaluation

The authors of Llava-Next forked ElutherAI's evaluation harness and built on top of it. That project is called [The Evaluation Suite of Large Multimodal Models](https://github.com/EvolvingLMMs-Lab/lmms-eval/tree/main). It's a bit hacky at the moment, but I think it's a step in the right direction.

I'm [adapting the code](https://github.com/EvolvingLMMs-Lab/lmms-eval/blob/main/lmms_eval/tasks/textvqa/utils.py) that the authors used for evaluation to better suit our setup.

In [48]:
from utils import add_accuracy_score

In [ ]:
columns_to_evaluate = ["mistral_answer", "vicuna_answer"]
textvqa_val_subset = textvqa_val_subset.map(add_accuracy_score, fn_kwargs={"columns_to_evaluate": columns_to_evaluate})

Let's take a quick look at the results.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = textvqa_val_subset.to_pandas()

means = df[['mistral_answer_score', 'vicuna_answer_score']].mean()
std_errs = df[['mistral_answer_score', 'vicuna_answer_score']].sem()
error = std_errs

# Creating the bar plot
fig, ax = plt.subplots()
means.plot(kind='bar', yerr=error, capsize=4, ax=ax, color=['#1f77b4', '#ff7f0e'], rot=0)

ax.set_ylabel('Scores')
ax.set_title('Average Scores with Error Bars')
ax.set_xticklabels(['Mistral Answer Score', 'Vicuna Answer Score'])

plt.tight_layout()
plt.show()


# Visualzing results in `fiftyone`!

It's definitley a close call for both models! But, looking at aggregrate metrics doesn't tell the whole story. It's not satisifying enough.

That's where `fiftyone` comes in.

Now that we've run inference and evaluation, let's massage our dataset into [`fiftyone`](https://github.com/voxel51/fiftyone) format so that we can visualize it in the `fiftyone` app.

In the app we can easily visually inspect the behavior of our models. We can see where the models agree, where they disagree, and where they differ from the ground truth.

In [ ]:
import fiftyone as fo
import datasets
import os
import PIL

def _get_extension(image):
    if isinstance(image, PIL.PngImagePlugin.PngImageFile):
        return ".png"
    elif isinstance(image, PIL.JpegImagePlugin.JpegImageFile):
        return ".jpg"
    else:
        return "web"

def load_textvqa_dataset_in_fiftyone(
        hf_dataset=textvqa_val_subset,
        download_dir='/content/textvqa_subset',
        name="textvqa"):

    dataset = fo.Dataset(name=name, persistent=True, overwrite=True)

    samples = []
    for i, item in enumerate(hf_dataset):
        img = item['image']
        ext = _get_extension(img)
        fp = os.path.join(download_dir, f'{i}{ext}')
        if not os.path.exists(fp):
            img.save(fp)

        sample_dict = {
        "filepath": fp,
        "tags": item['image_classes'],
        "question": item['question'],
        "acceptable_answers": list(set(item['answers'])),
        "vicuna_answer": item['vicuna_answer'],
        'mistral_answer': item['mistral_answer'],
        'mistral_answer_score': item['mistral_answer_score'],
        'vicuna_answer_score': item['vicuna_answer_score'],
        'image_classes': item['image_classes'],
        }

        sample = fo.Sample(**sample_dict)
        samples.append(sample)

    dataset.add_samples(samples)

    return dataset

In [ ]:
textvqa_test_fo = load_textvqa_dataset_in_fiftyone()

With our data in `fiftyone` format, we can visually inspect how the models perform.

Notice on the side panel you can filter to

In [54]:
session = fo.launch_app(textvqa_test_fo)